In [0]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 4.8MB/s 
     |████████████████████████████████| 655kB 45.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
Successfully built regex


In [0]:
from pytorch_pretrained_bert import BertTokenizer, BertForMaskedLM
import torch

CLS_TOKEN = ['[CLS]']
SEP_TOKEN = ['[SEP]']
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
model.eval()

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
100%|██████████| 662804195/662804195 [00:20<00:00, 31937304.58B/s]


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertInter

In [0]:
'''Fill question in any desired form. Fill answer with the following format: answer body [MASK] answer body.'''
'''For examle: What is the capital of Japan? The capital of Japan is [MASK] . '''
'''Dots are important! Spacing is important!'''

#question = 'How old is Abe Shinzo？'
#answer = 'Shinzo Abe is [MASK] old. '

#question = 'Какая столица России？'
#answer = ' Cтолица России - [MASK] . '

#question = 'What is the capital of Great Britain？'
#answer = 'The capital of Great Britain is [MASK] . '

question = '犬とは何です ？'
answer = '犬は [MASK] [MASK] です。'


In [0]:
tokenized_question = CLS_TOKEN + tokenizer.tokenize(question) + SEP_TOKEN
tokenized_answer = tokenizer.tokenize(answer) + SEP_TOKEN
tokenized_query = tokenized_question + tokenized_answer
mask_index = tokenized_query.index('[MASK]')

indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_query)
segments_ids = [0] * len(tokenized_question) + [1] * len(tokenized_answer)

In [0]:
print(tokenized_query)
print(segments_ids)
print(indexed_tokens)
assert len(segments_ids) == len(tokenized_query)

['[CLS]', '犬', 'とは', '何', 'で', '##す', '？', '[SEP]', '犬', 'は', '[MASK]', '[MASK]', 'で', '##す', '。', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
[101, 5420, 15904, 2253, 1939, 22445, 10083, 102, 5420, 1947, 103, 103, 1939, 22445, 1882, 102]


In [0]:
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

with torch.no_grad():
    predictions = model(tokens_tensor, segments_tensors)

In [0]:
#FIRST TOKEN
predicted_index = torch.argmax(predictions[0, mask_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token)

何


In [0]:
#SECOND TOKEN (IF PRESENT)
predicted_index = torch.argmax(predictions[0, mask_index+1]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token)

何
